# DL Survival - Ventilation Outcomes
 Updated 21/11/21

In [1]:
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
import datetime as dt
from datetime import timedelta
import json
import miceforest as mf
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

## 1. Data cleaning

- Import MIMIC III data
- Review column unique values, assign correct data types
- Impute missing values


### 1.1: Importing data

In [2]:
df = pd.read_csv('mimic_combined.csv')

#### 1.1.1: Column lists

In [3]:
#view and reorder columns
cols = list(df.columns)
new_cols = ['Unnamed: 0','hadm_id','subject_id','gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index','pt','ptt',
            'inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
            'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time','albumin',
            'creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate','po2','pco2','baseexcess','ph','aado2',
            'fio2','ffp','insulin','cryo','prbc','infection','ventrate','tidalvol','vent_array','reintubation','liver_severe','liver_mild',
            'rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi',
            'dementia','first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag','admittime',
            'dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime','plt','diab_un','diab_cc',
            'dtoutput','specimen','dod']

ptinfo=['Unnamed: 0','hadm_id','subject_id']

demographics=['gender','ethnicity','marital_status','insurance','language']

proceduretype=['aortic','mit','tricuspid','pulmonary','cabg']

vitals=['temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index']

labs=['pt','ptt','inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time',
'albumin','creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate']

bloodgases=['po2','pco2','baseexcess','ph','aado2','fio2']

products=['ffp','insulin','cryo','prbc','infection']

ventilation=['ventrate','tidalvol','vent_array','reintubation']

comorbidities=['liver_severe','liver_mild','rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd',
'paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi','dementia']

adm_cat=['first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag']

adm_num=['admittime','dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime']

others=['plt','diab_un','diab_cc','dtoutput','specimen','dod']

timeseries=[*vitals,*labs,*bloodgases,*products,*ventilation,'plt','dtoutput']
timeseries = [i for i in timeseries if i not in ('weight','height','reintubation', 'infection', 'vent_array')]
    
timeseries_valuenames = {'cardiac_index':'ci',
                         'plts':'bloodproduct',
                         'ffp':'bloodproduct',
                         'insulin':'amount',
                         'cryo':'bloodproduct',
                         'prbc':'bloodproduct',
                         'dtoutput':'output'}

In [4]:
df = df[new_cols]
df.head(10)

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod
0,0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.50,167.64,[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],[],[],[],[],"[{'suspected_infection_time': None, 'antibioti...",[],[],"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYS REFERRAL/NORMAL DELI,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaN,1.0424,1,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,[],[],NaN
1,1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.00,157.48,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetim

### 1.2: Cleaning data types

#### 1.2.0: NaN assignment

In [5]:
df = df.replace('NaT',np.datetime64('NaT'))
df = df.replace(['[]','NaN',np.datetime64('NaT')],np.NaN)

#### 1.2.1: Datetime columns

In [6]:
# set column types as datetime
time_cols = ['admittime','dischtime','intime','outtime','reint_time','ext_time','deathtime']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

#dod
df['dod'] = pd.to_datetime(df['dod'], format='%Y-%m-%d')

In [7]:
## CHECK FOR ROWS WHERE DEATHTIME < INTIME OR ADMITTIME

In [8]:
df[time_cols].dtypes

admittime     datetime64[ns]
dischtime     datetime64[ns]
intime        datetime64[ns]
outtime       datetime64[ns]
reint_time    datetime64[ns]
ext_time      datetime64[ns]
deathtime     datetime64[ns]
dtype: object

#### 1.2.2: Demographics

In [9]:
for x in demographics:
    print(x,': ',df[x].unique())

gender :  ['M' 'F']
ethnicity :  ['white' 'unknown' 'other' 'black' 'asian' 'hispanic' 'native' 'WHITE'
 'OTHER' 'UNKNOWN' 'UNABLE TO OBTAIN' 'ASIAN' 'HISPANIC/LATINO'
 'BLACK/AFRICAN AMERICAN' 'AMERICAN INDIAN/ALASKA NATIVE']
marital_status :  ['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'UNKNOWN (DEFAULT)'
 'SEPARATED']
insurance :  ['Private' 'Medicare' 'Medicaid' 'Government' 'Self Pay' 'Other']
language :  ['ENGL' nan 'CAPE' 'RUSS' 'MAND' 'PORT' 'CAMB' 'SPAN' 'PTUN' 'CANT' 'ITAL'
 'POLI' 'PERS' 'VIET' '*GUJ' 'URDU' '*BEN' 'KORE' 'ALBA' '*LEB' 'AMER'
 'GREE' 'HAIT' 'THAI' 'HIND' 'TURK' 'GERM' 'ARAB' 'ETHI' 'ENGLISH' '?']


In [10]:
#ethnicity
df.replace({'ethnicity':
                {'unknown': np.NaN,'UNKNOWN':np.NaN,'UNABLE TO OBTAIN':np.NaN,
                'OTHER':'other','WHITE':'white','BLACK/AFRICAN AMERICAN':'black','ASIAN':'asian',
                'HISPANIC/LATINO':'hispanic','AMERICAN INDIAN/ALASKA NATIVE':'native'
                }
            }, 
            inplace=True)
print(df['ethnicity'].unique())

['white' nan 'other' 'black' 'asian' 'hispanic' 'native']


In [11]:
#marital_status
df.replace({'marital_status':
                {'UNKNOWN (DEFAULT)': np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'SEPARATED']


In [12]:
#language
df.replace({'language':
                {'ENGLISH':'ENGL','?':np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'SEPARATED']


#### 1.2.3: ✔Procedure type

In [13]:
for x in proceduretype:
    print(x,': ',df[x].unique())

aortic :  [1 0]
mit :  [0 1]
tricuspid :  [0 1]
pulmonary :  [0 1]
cabg :  [0 1]


#### 1.2.4: **Vitals / Blood Gases / Products + infection / Ventilation


In [14]:
# wait for Jahan/others
# ventrate seems to be empty

#### 1.2.5: ✔Comorbidities

In [15]:
for x in comorbidities:
    print(x,': ',df[x].unique())

liver_severe :  [0 1]
liver_mild :  [0 1]
rheum :  [0 1]
cvd :  [0 1]
aids :  [0 1]
ckd :  [0 1]
copd :  [0 1]
arrhythmia :  [0 1]
pud :  [0 1]
smoking :  [0 1]
pvd :  [0 1]
paraplegia :  [0 1]
ccf :  [0 1]
met_ca :  [0 1]
t2dm :  [0 1]
t1dm :  [0 1]
malig :  [0 1]
mi :  [0 1]
dementia :  [0 1]


#### 1.2.6: Admissions (categorical)

In [16]:
for x in adm_cat:
    print(x,': ',df[x].unique())

first_careunit :  ['CSRU' 'CCU' 'SICU' 'MICU' 'TSICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)' 'Trauma SICU (TSICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)' 'Neuro Intermediate']
last_careunit :  ['CSRU' 'TSICU' 'CCU' 'MICU' 'SICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)' 'Trauma SICU (TSICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)']
admission_location :  ['PHYS REFERRAL/NORMAL DELI' 'TRANSFER FROM HOSP/EXTRAM'
 'CLINIC REFERRAL/PREMATURE' 'EMERGENCY ROOM ADMIT'
 'TRANSFER FROM OTHER HEALT' 'TRANSFER FROM SKILLED NUR'
 'PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM'
 'PROCEDURE SITE' 'WALK-IN/SELF REFERRAL' 'PACU'
 'INFORMATION N

In [17]:
#first_careunit
df.replace({'first_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['first_careunit'].unique())

['CSRU' 'CCU' 'SICU' 'MICU' 'TSICU' 'CVICU' 'MICU/SICU' 'Neuro SICU'
 'Neuro Inter']


In [18]:
#last_careunit
df.replace({'last_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['last_careunit'].unique())

['CSRU' 'TSICU' 'CCU' 'MICU' 'SICU' 'CVICU' 'MICU/SICU' 'Neuro SICU']


In [19]:
#admission_location
df.replace({'admission_location':
                {'TRANSFER FROM HOSP/EXTRAM':'TRANSFER FROM HOSPITAL',
                'PHYS REFERRAL/NORMAL DELI':'PHYSICIAN REFERRAL',
                'TRANSFER FROM SKILLED NUR':'TRANSFER FROM SKILLED NURSING FACILITY',
                'INFORMATION NOT AVAILABLE':np.NaN,
                'CLINIC REFERRAL':'CLINIC REFERRAL/PREMATURE',
                'EMERGENCY ROOM ADMIT':'EMERGENCY ROOM',
                }
            }, 
            inplace=True)
print(df['admission_location'].unique())

['PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'CLINIC REFERRAL/PREMATURE'
 'EMERGENCY ROOM' 'TRANSFER FROM OTHER HEALT'
 'TRANSFER FROM SKILLED NURSING FACILITY' 'PROCEDURE SITE'
 'WALK-IN/SELF REFERRAL' 'PACU' nan 'AMBULATORY SURGERY TRANSFER'
 'INTERNAL TRANSFER TO OR FROM PSYCH']


#### 1.2.7: Others

In [20]:
# for x in others:
#     print(x,': ',df[x].unique())

### 1.3: Parsing time series data

In [21]:
df['vent_array'][14]

"[{'starttime': datetime.datetime(2138, 7, 25, 18, 0), 'endtime': datetime.datetime(2138, 7, 28, 6, 0), 'duration_hours': 60.0}\n {'starttime': datetime.datetime(2138, 7, 29, 12, 0), 'endtime': datetime.datetime(2138, 8, 5, 11, 30), 'duration_hours': 167.5}]"

In [22]:
def va_parser(row, output=6):
    """
    Takes row from `df` returns a list of starttime, endtime, vent duration 
    for first and (if applicable) second intubations

    Parameters
    ----------
    row : row in df
    output_ : select which output you want (use list index below) - e.g. args=[6] for all output when using df.apply()

    Returns
    -------
    single list variable containing  
        [0] int_time1: first intubation starttime
        [1] ext_time1: first intubation endtime
        [2] duration1: first intubation duration
        [3] int_time2: second intubation starttime
        [4] ext_time2: second intubation endtime 
        [5] duration2: second intubation duration
        [6] all

    """
    int_time1=np.NaN
    ext_time1=np.NaN
    duration1=np.NaN
    int_time2=np.NaN
    ext_time2=np.NaN
    duration2=np.NaN
    value = row['vent_array']
    list=[]
    '''a = value
    print(value)'''
    if value == np.NaN or pd.isna(value):
        return np.NaN
    a = value.replace("'",'"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.','"dt.')
    a = a.replace('),', ')",')
    a = json.loads(a)
    b = [(i['starttime'], i['endtime'], i['duration_hours']) for i in a]
    int_time1=dt.datetime.strptime(b[0][0],'dt.datetime(%Y, %m, %d, %H, %M)')
    ext_time1=dt.datetime.strptime(b[0][1],'dt.datetime(%Y, %m, %d, %H, %M)')
    duration1=b[0][2]
    
    if output==0:
        return int_time1
    if output==1:
        return ext_time1
    if output==2:
        return duration1

    if len(b)>=2:
        int_time2=dt.datetime.strptime(b[1][0],'dt.datetime(%Y, %m, %d, %H, %M)')
        ext_time2=dt.datetime.strptime(b[1][1],'dt.datetime(%Y, %m, %d, %H, %M)')
        duration2=b[1][2]
    if output==3:
        return int_time2
    if output==4:
        return ext_time2
    if output==5:
        return duration2
    if output==6:
        return int_time1, ext_time1, duration1, int_time2, ext_time2, duration2

In [23]:
df['int_time1']=df.apply(va_parser, args=[0], axis=1)
df['ext_time1']=df.apply(va_parser, args=[1], axis=1)
df['duration1']=df.apply(va_parser, args=[2], axis=1)
df['int_time2']=df.apply(va_parser, args=[3], axis=1)
df['ext_time2']=df.apply(va_parser, args=[4], axis=1)
df['duration2']=df.apply(va_parser, args=[5], axis=1)
df.head()

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.5,167.64,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'suspected_infection_time': None, 'antibioti...",NaN,NaN,"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaT,1.0424,1,NaT,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,NaN,NaN,NaT,2111-02-16 10:00:00,2111-02-16 13:00:00,3.0,NaT,NaT,NaN
1,1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.0,157.48,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.date

In [24]:
def inf_parser(value, timeLimits=None):
    if value == np.NaN or pd.isna(value):
        return 0
    a = value.replace("'", '"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.', '"dt.')
    a = a.replace('": None', '": "None"')
    for valuename in ['antibiotic','antibiotic_time']:
        a = a.replace(f'), "{valuename}"', f')", "{valuename}"')
    a = json.loads(a)
    b = [(eval(i['suspected_infection_time']), eval(i['antibiotic_time']), i['antibiotic'], i['specimen'], 
               i['positiveculture']) for i in a]
    sus = [i[0] for i in b]
    abx = [i[1] for i in b]
    pos = [i[4] for i in b]
    inf_ = 0
    for i in range(len(sus)):
        if sus[i]==None:
            sus[i]=dt.datetime(1000, 1, 1, 0, 0)
        if abx[i]==None:
            abx[i]=dt.datetime(1000, 1, 1, 0, 0)
        if ((sus[i]>= timeLimits[0] and sus[i]<=timeLimits[1]) or (abx[i]>= timeLimits[0] and abx[i]<=timeLimits[1])) and pos[i] == 1.0:
            temp = 1
            break
        else:
            continue
    return inf_

In [25]:
def ts_parser(value, timeDelta=None, timeLimits=None, valuename='value'):
    # timeDelta is timedelta in hours from earliest entry
    # timeLimits = (startTime, endTime)
    # if both timeDelta and timeLimits are provided, timeDelta overrules.
    # if both are None, then all timepoints are accepted
    if value == np.NaN or pd.isna(value):
        return np.NaN, np.NaN, np.NaN
    a = value.replace("'", '"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.', '"dt.')
    a = a.replace(f'), "{valuename}"', f')", "{valuename}"')
    a = a.replace('"unit": None', '"unit": "None"')
    a = a.replace('starttime', 'charttime')
    a = json.loads(a)
    b = [(eval(i['charttime']), i[valuename]) for i in a]
    
    if timeDelta:
        startTime = min(b, key=lambda x:x[0])[0]
        inc_b = [i[1] for i in b if i[0] <= startTime + dt.timedelta(hours=timeDelta)]
    else:
        if timeLimits:
            inc_b = [i[1] for i in b if i[0] >= timeLimits[0] and i[0] <= timeLimits[1]]
        else:
            inc_b = [i[1] for i in b]
    if len(inc_b) == 0:
        return np.NaN, np.NaN, np.NaN
    
    return sum(inc_b) / len(inc_b), max(inc_b), min(inc_b)

In [26]:
# delete all rows where int_time1 OR ext_time1 are missing
df = df[~(pd.isnull(df['int_time1']) | pd.isnull(df['ext_time1']))]
df = df.reset_index()
df = df.drop(axis=1, columns=['index', 'Unnamed: 0'], inplace=False)
df

,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.50,167.64,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'suspected_infection_time': None, 'antibioti...",NaN,NaN,"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaT,1.042400,1,NaT,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,NaN,NaN,NaT,2111-02-16 10:00:00,2111-02-16 13:00:00,3.0,NaT,NaT,NaN
1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.00,157.48,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 

### 1.4: Handling missing data

#### 1.4.0 Assessing for missing data

In [27]:
# check % missing values
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: 'Missing Values', 1: '% Missing Values'})
    return mis_val_table_ren_columns

missing_data = missing_values_table(df)

In [28]:
#set limit and get list of variables missing above limit in `missing_cols`
# missing_limit = 50 #allen
missing_limit = 80 #marcel
missing_cols = missing_data.loc[missing_data['% Missing Values']>missing_limit].index.tolist()
print(missing_cols)
missing_data = missing_data.loc[missing_data['% Missing Values']>missing_limit]
missing_data = missing_data.sort_values(by=['% Missing Values'])
pd.set_option('display.max_rows', None)
missing_data

['plts', 'ggt', 'bilirubin_indirect', 'bilirubin_direct', 'crp', 'bleed_time', 'albumin', 'aado2', 'ffp', 'cryo', 'ventrate', 'reint_time', 'deathtime', 'dtoutput', 'specimen', 'dod', 'int_time2', 'ext_time2', 'duration2']


,Missing Values,% Missing Values
plts,7785,82.172261
albumin,7879,83.164450
ffp,7928,83.681655
aado2,8018,84.631623
dod,8059,85.064387
cryo,8567,90.426430
ext_time2,8845,93.360777
reint_time,8845,93.360777
int_time2,8845,93.360777
duration2,8845,93.360777


In [29]:
pd.reset_option('display.max_rows')

In [30]:
#df = df.drop(axis=1, columns=list(missing_data.index), inplace=False) #allen
df = df.drop(axis=1, columns=[i for i in list(missing_data.index) if i not in ['duration2','int_time2','ext_time2','aado2','fio2']], inplace=False) #marcel
print(list(df.columns))

['hadm_id', 'subject_id', 'gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'aortic', 'mit', 'tricuspid', 'pulmonary', 'cabg', 'temp', 'bg_temp', 'hr', 'spo2', 'rr', 'sbp', 'dbp', 'meanbp', 'weight', 'height', 'cardiac_index', 'pt', 'ptt', 'inr', 'inr_1', 'fibrinogen', 'hb', 'hematocrit', 'wcc', 'lymphocytes', 'neutrophils', 'alp', 'ast', 'alt', 'bilirubin_total', 'chloride', 'magnesium', 'potassium', 'creatinine', 'free_calcium', 'sodium', 'bicarb', 'bun', 'hba1c', 'glucose', 'lactate', 'po2', 'pco2', 'baseexcess', 'ph', 'aado2', 'fio2', 'insulin', 'prbc', 'infection', 'tidalvol', 'vent_array', 'reintubation', 'liver_severe', 'liver_mild', 'rheum', 'cvd', 'aids', 'ckd', 'copd', 'arrhythmia', 'pud', 'smoking', 'pvd', 'paraplegia', 'ccf', 'met_ca', 't2dm', 't1dm', 'malig', 'mi', 'dementia', 'first_careunit', 'last_careunit', 'admission_location', 'admission_type', 'hospital_expire_flag', 'admittime', 'dischtime', 'intime', 'outtime', 'ext_time', 'los', 'icustay_seq', 'pl

#### 1.4.1 Beginning imputation

In [31]:
dfForImpute = pd.DataFrame([0 for i in range(df.shape[0])])

# generating timeseries summary values
for column in timeseries:
    if column not in df.columns:
        continue
    x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
    meanList = []
    maxList = []
    minList = []
    for i in range(len(df[column])):
        y = ts_parser(df[column][i], timeLimits=(df['int_time1'][i].to_pydatetime(), df['ext_time1'][i].to_pydatetime()), valuename=x)
        meanList.append(y[0])
        maxList.append(y[1])
        minList.append(y[2])
    dfForImpute[column+'_mean'] = meanList
    dfForImpute[column+'_max'] = maxList
    dfForImpute[column+'_min'] = minList

C:\Users\marcel\AppData\Local\Temp/ipykernel_14792/247845895.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute[column+'_mean'] = meanList
C:\Users\marcel\AppData\Local\Temp/ipykernel_14792/247845895.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute[column+'_max'] = maxList
C:\Users\marcel\AppData\Local\Temp/ipykernel_14792/247845895.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [32]:
# generating infection during ventilation binary values

x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
infList = []
column='infection'
for i in range(len(df[column])):
    y = inf_parser(df[column][i], timeLimits=(df['int_time1'][i].to_pydatetime(), df['ext_time1'][i].to_pydatetime()))
    infList.append(y)
dfForImpute['infection_vent'] = infList

C:\Users\marcel\AppData\Local\Temp/ipykernel_14792/502382886.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute['infection_vent'] = infList


In [39]:
# add on non-time data for imputation

#extraColumns = [i for i in df.columns if i not in list(dfForImpute.columns) + timeseries + ['infection', 'vent_array', 'int_time1', 'ext_time1'] + ptinfo + adm_num] #allen
extraColumns = [i for i in df.columns if i not in list(dfForImpute.columns) + timeseries + ['vent_array','infection'] + ptinfo + adm_num] #marcel

for i in extraColumns:
    if i in ('weight', 'height', 'duration1','duration2'):
        dfForImpute[i] = df[i]
    else:
        dfForImpute[i] = df[i].astype('category')
dfForImpute.drop(axis=1, columns=[0], inplace=True)
dfForImpute2 = dfForImpute.copy()

In [40]:
# before imputation again
dfForImpute

,temp_mean,temp_max,temp_min,bg_temp_mean,bg_temp_max,bg_temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,cardiac_index_mean,cardiac_index_max,cardiac_index_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,lymphocytes_mean,lymphocytes_max,lymphocytes_min,neutrophils_mean,neutrophils_max,neutrophils_min,alp_mean,alp_max,alp_min,ast_mean,ast_max,ast_min,alt_mean,alt_max,alt_min,bilirubin_total_mean,bilirubin_total_max,bilirubin_total_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,aado2_mean,aado2_max,aado2_min,fio2_mean,fio2_max,fio2_min,insulin_mean,insulin_max,insulin_min,prbc_mean,prbc_max,prbc_min,tidalvol_mean,tidalvol_max,tidalvol_min,plt_mean,plt_max,plt_min,infection_vent,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,36.031745,36.444444,35.611110,NaN,NaN,NaN,89.000000,98.0,79.0,100.000000,100.0,100.0,14.230769,16.0,14.0,111.166667,123.0,90.0,54.750000,59.0,44.0,68.333333,74.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.900000,3.9,3.9,NaN,NaN,NaN,1.2800,1.28,1.28,137.0,137.0,137.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.000000,159.0,79.0,NaN,NaN,NaN,2.000000,2.0,2.0,45.000000,45.0,45.0,2.000000,2.0,2.0,7.390000,7.39,7.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,69.50,167.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,2111-02-16 10:00:00,2111-02-16 13:00:00,3.0,NaT,NaT,NaN
1,36.376923,36.500000,36.099998,NaN,NaN,NaN,84.538462,90.0,78.0,99.923077,100.0,99.0,15.576923,24.5,11.0,106.384615,123.0,81.0,51.000000,59.0,41.0,65.538462,76.0,52.0,2.293039,4.05128,1.84615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.200000,4.2,4.2,NaN,NaN,NaN,1.2700,1.28,1.26,136.5,137.0,136.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.500000,103.0,98.0,NaN,NaN,NaN,0.000000,0.0,0.0,44.000000,47.0,41.0,0.000000,0.0,0.0,7.365000,7.38,7.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,82.00,157.48,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,2192-04-20 12:00:00,2192-04-20 15:00:00,3.0,NaT,NaT,NaN
2,36.235714,36.799999,35.400002,36.3,36.6,36.0,84.910714,93.0,71.0,98.785714,100.0,96.0,19.071429,38.0,10.0,125.196429,194.0,76.0,54.642857,73.0,40.0,78.053571,113.0,53.0,2.346591,3.35227,1.35795,NaN,NaN,NaN,46.133333,73.2,32.2,1.60,2.0,1.3,1.60,2.0,1.3,NaN,NaN,NaN,12.450000,12.7,12.2,38.0,38.0,38.0,20.7,20.7,20.7,NaN,NaN,NaN,NaN,NaN,NaN,49.0,49.0,49.0,81.0,81.0,81.0,33.0,33.0,33.0,0.5,0.5,0.5,

In [43]:
dfForImpute2 = dfForImpute.drop(axis=1, columns=['int_time1', 'ext_time1', 'duration1','int_time2', 'ext_time2','duration2'])

kds = mf.ImputationKernel(
  dfForImpute2,
  datasets=1,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations
kds.mice(3)

print(kds)

dfImputed = kds.complete_data(dataset=0, inplace=False)
print(dfImputed.isnull().sum(0))

# after imputation
dfImputed.head(10)

              Class: ImputationKernel
           Datasets: 1
         Iterations: 3
  Imputed Variables: 138
save_all_iterations: True
temp_mean               0
temp_max                0
temp_min                0
bg_temp_mean            0
bg_temp_max             0
                       ..
admission_location      0
admission_type          0
hospital_expire_flag    0
diab_un                 0
diab_cc                 0
Length: 172, dtype: int64


,temp_mean,temp_max,temp_min,bg_temp_mean,bg_temp_max,bg_temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,cardiac_index_mean,cardiac_index_max,cardiac_index_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,lymphocytes_mean,lymphocytes_max,lymphocytes_min,neutrophils_mean,neutrophils_max,neutrophils_min,alp_mean,alp_max,alp_min,ast_mean,ast_max,ast_min,alt_mean,alt_max,alt_min,bilirubin_total_mean,bilirubin_total_max,bilirubin_total_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,aado2_mean,aado2_max,aado2_min,fio2_mean,fio2_max,fio2_min,insulin_mean,insulin_max,insulin_min,prbc_mean,prbc_max,prbc_min,tidalvol_mean,tidalvol_max,tidalvol_min,plt_mean,plt_max,plt_min,infection_vent,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc
0,36.031745,36.444444,35.611110,37.450000,37.8,37.0,89.000000,98.0,79.0,100.000000,100.0,100.0,14.230769,16.0,14.0,111.166667,123.0,90.0,54.750000,59.0,44.0,68.333333,74.0,55.0,3.370576,3.92350,2.78488,14.4500,15.0,14.3,28.850000,29.1,29.2,1.283333,1.3,1.3,1.290000,1.3,1.3,136.000000,139.0,138.0,11.050000,11.0,11.2,32.666667,33.0,32.0,16.100000,17.5,14.8,10.40,13.3,8.5,85.0,86.0,83.7,100.5,109.0,85.0,25.000000,26.0,28.0,17.750000,18.0,17.0,0.500000,0.5,0.5,110.500000,112.0,109.0,1.900000,2.1,1.7,3.900000,3.9,3.9,0.90,1.0,0.8,1.280000,1.28,1.28,137.000000,137.0,137.0,23.000000,23.0,23.0,16.666667,18.0,15.0,6.40,6.4,6.1,95.000000,159.0,79.0,1.950000,2.1,1.9,2.000000,2.0,2.0,45.000000,45.0,45.0,2.000000,2.0,2.0,7.390000,7.39,7.39,482.00,490.0,478.0,46.666667,50.0,40.0,202.666667,291.0,184.000000,466.666673,800.0,275.00001,800.000000,800.0,800.0,143.000000,153.0,129.0,0,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,69.50,167.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0
1,36.376923,36.500000,36.099998,36.675000,37.0,36.3,84.538462,90.0,78.0,99.923077,100.0,99.0,15.576923,24.5,11.0,106.384615,123.0,81.0,51.000000,59.0,41.0,65.538462,76.0,52.0,2.293039,4.05128,1.84615,15.8000,16.4,15.6,38.700000,38.7,39.2,1.400000,1.5,1.4,1.400000,1.5,1.4,179.000000,179.0,179.0,12.350000,13.1,11.7,30.500000,29.0,30.0,6.800000,6.9,6.4,9.80,12.5,7.5,69.2,69.4,69.0,53.0,54.0,52.0,82.000000,90.0,74.0,70.000000,70.0,65.0,0.650000,0.7,0.6,107.666667,111.0,105.0,1.980000,2.2,1.9,4.200000,4.2,4.2,0.70,0.7,0.7,1.270000,1.28,1.26,136.500000,137.0,136.0,23.000000,24.0,22.0,13.000000,14.0,12.0,6.00,6.0,6.0,100.500000,103.0,98.0,2.700000,2.7,2.1,0.000000,0.0,0.0,44.000000,47.0,41.0,0.000000,0.0,0.0,7.365000,7.38,7.35,352.00,357.0,356.0,50.000000,50.0,50.0,232.400000,357.0,0.000000,300.000000,551.0,0.00000,800.000000,800.0,800.0,117.333333,135.0,120.0,0,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,82.00,157.48,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0
2,36.235714,36.799999,35.400002,36.300000,36.6,36.0,84.910714,93.0,71.0,98.785714,100.0,96.0,19.071429,38.0,10.0,125.196429,194.0,76.0,54.64

In [45]:
y = np.asarray([i.to_pydatetime() for i in df["int_time1"]])
z = np.asarray([i.to_pydatetime() for i in df["outtime"]])
dfImputed['icu_stay_duration'] = [i.total_seconds() for i in z-y]
dfImputed[['hadm_id','subject_id']] = df[['hadm_id','subject_id']]
dfImputed[['int_time1', 'ext_time1', 'duration1','int_time2', 'ext_time2','duration2']] = df[['int_time1', 'ext_time1', 'duration1','int_time2', 'ext_time2','duration2']] #marcel not #allen
dfImputed

,temp_mean,temp_max,temp_min,bg_temp_mean,bg_temp_max,bg_temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,cardiac_index_mean,cardiac_index_max,cardiac_index_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,lymphocytes_mean,lymphocytes_max,lymphocytes_min,neutrophils_mean,neutrophils_max,neutrophils_min,alp_mean,alp_max,alp_min,ast_mean,ast_max,ast_min,alt_mean,alt_max,alt_min,bilirubin_total_mean,bilirubin_total_max,bilirubin_total_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,aado2_mean,aado2_max,aado2_min,fio2_mean,fio2_max,fio2_min,insulin_mean,insulin_max,insulin_min,prbc_mean,prbc_max,prbc_min,tidalvol_mean,tidalvol_max,tidalvol_min,plt_mean,plt_max,plt_min,infection_vent,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,icu_stay_duration,hadm_id,subject_id,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,36.031745,36.444444,35.611110,37.450000,37.8,37.0,89.000000,98.0,79.0,100.000000,100.0,100.0,14.230769,16.0,14.0,111.166667,123.0,90.0,54.750000,59.0,44.0,68.333333,74.0,55.0,3.370576,3.92350,2.78488,14.450000,15.0,14.3,28.850000,29.1,29.2,1.283333,1.3,1.3,1.29,1.3,1.3,136.0,139.0,138.0,11.050000,11.0,11.2,32.666667,33.0,32.0,16.10,17.5,14.8,10.400000,13.3,8.5,85.00,86.0,83.7,100.500000,109.0,85.0,25.00,26.0,28.0,17.750000,18.0,17.0,0.500000,0.5,0.5,110.500000,112.0,109.0,1.900000,2.1,1.7,3.900000,3.9,3.9,0.90,1.0,0.8,1.2800,1.28,1.28,137.0,137.0,137.0,23.00,23.0,23.0,16.666667,18.0,15.0,6.40,6.4,6.1,95.000000,159.0,79.0,1.950000,2.1,1.9,2.000000,2.0,2.0,45.000000,45.0,45.0,2.000000,2.0,2.0,7.390000,7.39,7.39,482.00,490.0,478.0,46.666667,50.0,40.0,202.666667,291.000000,184.000000,466.666673,800.000000,275.000010,800.000000,800.0,800.0,143.000000,153.0,129.0,0,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,69.50,167.64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,91283.0,173798,29491,2111-02-16 10:00:00,2111-02-16 13:00:00,3.0,NaT,NaT,NaN
1,36.376923,36.500000,36.099998,36.675000,37.0,36.3,84.538462,90.0,78.0,99.923077,100.0,99.0,15.576923,24.5,11.0,106.384615,123.0,81.0,51.000000,59.0,41.0,65.538462,76.0,52.0,2.293039,4.05128,1.84615,15.800000,16.4,15.6,38.700000,38.7,39.2,1.400000,1.5,1.4,1.40,1.5,1.4,179.0,179.0,179.0,12.350000,13.1,11.7,30.500000,29.0,30.0,6.80,6.9,6.4,9.800000,12.5,7.5,69.20,69.4,69.0,53.000000,54.0,52.0,82.00,90.0,74.0,70.000000,70.0,65.0,0.650000,0.7,0.6,107.666667,111.0,105.0,1.980000,2.2,1.9,4.200000,4.2,4.2,0.70,0.7,0.7,1.2700,1.28,1.26,136.5,137.0,136.0,23.00,24.0,22.0,13.000000,14.0,12.0,6.00,6.0,6.0,100.500000,103.0,98.0,2.700000,2.7,2.1,0.000000,0.0,0.0,44.000000,47.0,41.0,0.000000,0.0,0.0,7.365000,7.38,7.35,352.00,357.0,356.0,50.000000,50.0,50.0,232.400000,357.000000,0.000000,300.000000,551.000000,0.000000,800.000000,800.0,800.0,117.333333,135.0,120.0,0,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,82.00,157.48,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN 

In [46]:
#dfImputed.to_csv('imputed.csv') #allen
dfImputed.to_csv('imputed_m.csv') #marcel